In [2]:
import plotly.express as px
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

In [76]:
# Main Timeline plot -------------------------------------------------------------------------------------------
alo_data = pd.read_csv('treated_data/alo_career_summary.csv')

alo_data['start.season'] = pd.to_datetime(alo_data['start.season'].astype(str), format='%Y')
alo_data['End.season'] = pd.to_datetime(alo_data['End.season'].astype(str), format='%Y')

color_map = {
    "Minardi": "black", 
    "Renault": "#fcd205", # renault yellow
    "McLaren": "#E56717", # papaya orange
    "Ferrari": "#F70D1A", # Ferrari red
    "Alpine": "#2673E2", # Alpine blue
    "Aston Martin": "#004225" # british racing green
}

text_color_map = {
    "Minardi": "gold",
    "Renault": "grey",
    "McLaren": "black",
    "Ferrari": "yellow",
    "Alpine": "white",
    "Aston Martin": "white"
}

# Assuming alo_data is already loaded and processed
current_year = datetime.now().year

# Create the Gantt chart with Plotly Express
fig = px.timeline(
    alo_data,
    x_start="start.season",
    x_end="End.season",
    y="Constructors",
    color="Constructors",
    color_discrete_map=color_map
)

# Convert Plotly Express figure to Plotly Graph Objects figure for custom adjustments
fig = go.Figure(fig)

# Adjusting the opacity for each trace based on the 'End.season' year
for trace in fig.data:
    end_years = alo_data[alo_data['Constructors'] == trace.name]['End.season'].dt.year
    opacities = [0.66 if year > 2024 else 1 for year in end_years]
    trace.marker.opacity = opacities

last_periods = alo_data.groupby('Constructors')['End.season'].max()

# Customize text color to contrast with the bar color and add annotations
for index, row in alo_data.iterrows():
    if row['End.season'] == last_periods[row['Constructors']]:
        # Calculate position for the annotation
        text_position = row['End.season'] - ((row['End.season'] - row['start.season']) / 10)
        fig.add_annotation(
            x=text_position, y=row['Constructors'],
            text=row['Constructors'],
            showarrow=False,
            font=dict(
                color=text_color_map[row['Constructors']],
                size=12,  # Adjust size as needed
                family='Calibri',  # You can specify the font family
                weight='bold'  # Making text bold
            ),
            xanchor='right',
            align='right')

# Reverse the y-axis
fig.update_yaxes(autorange="reversed")

# Hide the legend as it's redundant now
fig.update_layout(showlegend=False)

# Adding red lines for specific years
years_of_interest = ["2005-01-01", "2006-01-02"]
for year in years_of_interest:
    fig.add_shape(
        type="line",
        x0=year, x1=year,
        y0=-0.5, y1=len(alo_data['Constructors'].unique()) - 0.5,
        line=dict(color="red", width=1.5)
    )

# Adjust x-axis ticks to show every year and set an upper limit for the x-axis
fig.update_xaxes(
    tickmode='linear',
    dtick="M12",
    range=["2000-12-01", "2026-02-01"],
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgrey'
)
fig.update_layout(
    title='Timeline of teams where Alonso drove',  # Adding a title
    title_x=0.5,  # Center the title
    paper_bgcolor='white',  # Set the background color around the plot
    plot_bgcolor='white'  # Set the background color of the plot itself
)
fig.show()

In [6]:
# 2005 season ---------------------------------------------------------------------------------------------------
# The race for the title
season_2005 = pd.read_csv('treated_data/csv/results.csv')
season_2005 = season_2005[season_2005['season'] == 2005]
season_2005['points'] = pd.to_numeric(season_2005['points'], errors='coerce')
season_2005 = season_2005.sort_values(by=['raceId'])
season_2005['cumulative_points'] = season_2005.groupby('driverId')['points'].cumsum()
final_plot_data = season_2005[['driverId', 'raceId', 'cumulative_points']]
total_points_per_driver = season_2005.groupby('driverId')['points'].sum().reset_index().rename(columns={'points': 'points_total'})
final_plot_data = final_plot_data.merge(total_points_per_driver, on='driverId', how='left')
final_plot_data = final_plot_data.sort_values(by=['points_total', 'raceId'], ascending=[False, True])
sorted_drivers = total_points_per_driver['driverId'].tolist()
#save sorted drivers to csv
final_plot_data.to_csv('treated_data/csv/2005_sorted_drivers.csv', index=False)

driver_colors = {
    'fisichella': '#FFFF00',  # hex for yellow
    'barrichello': '#FF0000',  # hex for red
    'alonso': '#FFFF00',  # hex for yellow
    'coulthard': '#00008B',  # hex for dark blue
    'webber': '#ADD8E6',  # hex for light blue
    'montoya': '#FFA500',  # hex for orange
    'klien': '#00008B',  # hex for dark blue
    'raikkonen': '#FFA500',  # hex for orange
    'trulli': '#FFC0CB',  # hex for pink
    'massa': '#0000FF',  # hex for blue
    'button': '#D3D3D3',  # hex for light grey
    'ralf_schumacher': '#FFC0CB',  # hex for pink
    'villeneuve': '#0000FF',  # hex for blue
    'sato': '#D3D3D3',  # hex for light grey
    'karthikeyan': '#9ACD32',  # using yellowgreen, darkyellow doesn't exist
    'monteiro': '#9ACD32',  # using yellowgreen
    'friesacher': '#000000',  # hex for black
    'michael_schumacher': '#FF0000',  # hex for red
    'heidfeld': '#ADD8E6',  # hex for light blue
    'albers': '#000000'  # hex for black
}


# Dictionary for custom line styles per driver (solid by default, dash or dot as specified)
driver_line_styles = {
    'fisichella': 'dash',
    'barrichello': 'dash',
    'montoya': 'dash',
    'klien': 'dash',
    'trulli': 'dash',
    'massa': 'dash',
    'sato': 'dash',
    'karthikeyan': 'dash',
    'friesacher': 'dash',
    'heidfeld': 'dash',
    # Default is solid, not listed
}

final_plot_data['driverId'] = pd.Categorical(
    final_plot_data['driverId'],
    categories=sorted_drivers,
    ordered=True
)

# Create the plot using Plotly Express
fig = px.line(
    final_plot_data,
    x='raceId',
    y='cumulative_points',
    color='driverId',  # Ensures legend order
    labels={'driverId': 'Driver', 'raceId': 'Race', 'cumulative_points': 'Cumulative Points'}
)

# Convert to Graph Objects for detailed customization
fig = go.Figure(fig)

# Update colors and line styles based on custom definitions
for trace in fig.data:
    driver_name = trace.name.lower()  # Assuming driverId is used as name, adjust if necessary
    trace.line.color = driver_colors.get(driver_name, '#3b3b3b')  # Default to gray if not specified
    trace.line.dash = driver_line_styles.get(driver_name, 'solid')  # Default to solid if not specified

fig.update_layout(
    title='2005 Race for the Title',  # Adding a title
    title_x=0.5,  # Center the title
    paper_bgcolor='white',  # Set the background color around the plot
    plot_bgcolor='white'  # Set the background color of the plot itself
)
fig.update_xaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgrey'
)
# Show the figure
fig.show()


In [5]:
# 2005 season ---------------------------------------------------------------------------------------------------
# The race for the title
season_2006 = pd.read_csv('treated_data/csv/results.csv')
season_2006 = season_2006[season_2006['season'] == 2006]
season_2006['points'] = pd.to_numeric(season_2006['points'], errors='coerce')
season_2006 = season_2006.sort_values(by=['raceId'])
season_2006['cumulative_points'] = season_2006.groupby('driverId')['points'].cumsum()
final_plot_data = season_2006[['driverId', 'raceId', 'cumulative_points']]
total_points_per_driver = season_2006.groupby('driverId')['points'].sum().reset_index().rename(columns={'points': 'points_total'})
final_plot_data = final_plot_data.merge(total_points_per_driver, on='driverId', how='left')
final_plot_data = final_plot_data.sort_values(by=['points_total', 'raceId'], ascending=[False, True])
sorted_drivers = total_points_per_driver['driverId'].tolist()
final_plot_data.to_csv('treated_data/csv/2006_sorted_drivers.csv', index=False)
driver_colors = {
    'alonso': '#FFD700',  # Main driver for Renault
    'fisichella': '#FFD700',  # Second driver for Renault
    'michael_schumacher': '#DC0000',  # Main driver for Ferrari
    'massa': '#DC0000',  # Second driver for Ferrari
    'raikkonen': '#FF8700',  # Main driver for McLaren
    'montoya': '#FF8700',  # Second driver for McLaren
    'button': '#FFC0CB',  # Main driver for Honda
    'barrichello': '#FFC0CB',  # Second driver for Honda
    'webber': '#ADD8E6',  # Main driver for Williams
    'rosberg': '#ADD8E6',  # Second driver for Williams
    'coulthard': '#00008B',  # Main driver for Red Bull
    'klien': '#00008B',  # Second driver for Red Bull
    'ralf_schumacher': '#000000',  # Main driver for Toyota
    'trulli': '#000000',  # Second driver for Toyota
    'heidfeld': '#C8F4F6',  # Main driver for BMW Sauber
    'villeneuve': '#C8F4F6',  # Second driver for BMW Sauber
    'liuzzi': '#0000FF',  # Main driver for Toro Rosso
    'speed': '#0000FF',  # Second driver for Toro Rosso
    'monteiro': '#F75920',  # Main driver for MF1
    'albers': '#F75920',  # Second driver for MF1
    'sato': '#B4B4B4',  # Main driver for Super Aguri
    'ide': '#B4B4B4'  # Second driver for Super Aguri
}


# Dictionary for custom line styles per driver (solid by default, dash or dot as specified)
driver_line_styles = {
    'alonso': 'solid',  # Main driver for Renault
    'fisichella': 'dash',  # Second driver for Renault
    'michael_schumacher': 'solid',  # Main driver for Ferrari
    'massa': 'dash',  # Second driver for Ferrari
    'raikkonen': 'solid',  # Main driver for McLaren
    'montoya': 'dash',  # Second driver for McLaren
    'button': 'solid',  # Main driver for Honda
    'barrichello': 'dash',  # Second driver for Honda
    'webber': 'solid',  # Main driver for Williams
    'rosberg': 'dash',  # Second driver for Williams
    'coulthard': 'solid',  # Main driver for Red Bull
    'klien': 'dash',  # Second driver for Red Bull
    'ralf_schumacher': 'solid',  # Main driver for Toyota
    'trulli': 'dash',  # Second driver for Toyota
    'heidfeld': 'solid',  # Main driver for BMW Sauber
    'villeneuve': 'dash',  # Second driver for BMW Sauber
    'liuzzi': 'solid',  # Main driver for Toro Rosso
    'speed': 'dash',  # Second driver for Toro Rosso
    'monteiro': 'solid',  # Main driver for MF1
    'albers': 'dash',  # Second driver for MF1
    'sato': 'solid',  # Main driver for Super Aguri
    'ide': 'dash'  # Second driver for Super Aguri
}


final_plot_data['driverId'] = pd.Categorical(
    final_plot_data['driverId'],
    categories=sorted_drivers,
    ordered=True
)

# Create the plot using Plotly Express
fig = px.line(
    final_plot_data,
    x='raceId',
    y='cumulative_points',
    color='driverId',  # Ensures legend order
    labels={'driverId': 'Driver', 'raceId': 'Race', 'cumulative_points': 'Cumulative Points'},
    title='2006 Race for the Title'
)

# Convert to Graph Objects for detailed customization
fig = go.Figure(fig)

# Update colors and line styles based on custom definitions
for trace in fig.data:
    driver_name = trace.name.lower()  # Assuming driverId is used as name, adjust if necessary
    trace.line.color = driver_colors.get(driver_name, '#3b3b3b')  # Default to gray if not specified
    trace.line.dash = driver_line_styles.get(driver_name, 'solid')  # Default to solid if not specified

fig.update_layout(
    title='2006 Race for the Title',  # Adding a title
    title_x=0.5,  # Center the title
    paper_bgcolor='white',  # Set the background color around the plot
    plot_bgcolor='white'  # Set the background color of the plot itself
)
fig.update_xaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgrey'
)
# Show the figure
fig.show()


In [41]:
# A plot (the type of plot yet to be decided) with Alonso's career in numbers, so:
# Number of races (in total and per team)
# Wins(in total and per team)
# Poles(in total and per team)
# Podiums(in total and per team)
# Fastest Laps(in total and per team)
# WDC(in total and per team)
# And add buttons to select what teams to see

# get total of number of races for alonso
all_results = pd.read_csv('treated_data/csv/results.csv')
all_results = all_results[all_results['driverId'] == 'alonso']
all_wins = all_results[all_results['position'] == 1]
all_poles = all_results[all_results['grid'] == 1]
podiums = all_results[all_results['position'].isin([1, 2, 3])]
print(all_results['raceId'].count())
print(all_wins['raceId'].count())
print(all_poles['raceId'].count())
print(podiums['raceId'].count())
# do we have fastest laps in the data?

# plots total KM raced (per season, per team and total)
# an easy way to do this is match the circuitid form circuit_info.csv with the circuitId from results.csv and sum the lengths


386
32
22
106
